In [22]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
data = pd.read_csv("DelhiNCR Restaurants.csv.xls")
data

,Restaurant_Name,Category,Pricing_for_2,Locality,Dining_Rating,Dining_Review_Count,Delivery_Rating,Delivery_Rating_Count,Website,Address,Phone_No,Latitude,Longitude,Known_For2,Known_For22
0,Rustom's,"Parsi, Street Food, North Indian, Desserts, Be...",2100,"ITO, New Delhi",4.9,1885,4.4,1844,https://www.zomato.com/ncr/rustoms-ito-new-delhi,"Delhi Parsi Anjuman, LNJP Colony, Bahadur Shah...",9.1991E+11,28.638167,77.240472,NaN,NaN
1,Cafe Lota,"Cafe, South Indian, North Indian, Beverages",1200,"Pragati Maidan, New Delhi",4.9,3748,3.9,37,https://www.zomato.com/ncr/cafe-lota-pragati-m...,"National Crafts Museum, Gate 2, Bhairon Marg, ...",9.17839E+11,28.613429,77.242471,"Pondicherry Fish Curry, Coconut Rabdi, Sabudan...","Artistic Decor, The Service, Natural Ambience,..."
2,Dum-Pukht - ITC Maurya,"Mughlai, North Indian, Desserts",5000,"ITC Maurya, Chanakyapuri, New Delhi",4.9,1371,NaN,0,https://www.zomato.com/ncr/dum-pukht-itc-maury...,"ITC Maurya, Chanakyapuri, New Delhi",9.11146E+11,28.598200,77.173661,"Royal ambiance and Mughlai cuisine, also recei...",NaN
3,Burma Burma,"Asian, Burmese, Bubble Tea, Desserts, Salad",1600,"Cyber Hub, DLF Cyber City, Gurgaon",4.9,2636,4.4,1238,https://www.zomato.com/ncr/burma-burma-dlf-cyb...,"Shop 6, Ground Floor, Building 8, Tower C, Cyb...",9.11244E+11,28.494464,77.088532,"Coconut Milk Dessert, Honey Caviar, Kaffir Lim...","Knowledgeable Staff, Authentic, Soothing Ambie..."
4,The Big Chill,"Continental, American, Italian",1500,"Khan Market, New Delhi",4.9,6487,NaN,0,https://www.zomato.com/ncr/the-big-chill-khan-...,"68-A, Khan Market, New Delhi",9.11142E+11,28.600350,77.227493,"Irish Cream Tiramisu, Penne Vodka, Mississippi...","Retro Ambience, Yummy Desserts, Big Portions, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1960,Bablu Chic-Inn,"Mughlai, North Indian, Chinese, Fast Food",500,"Ashok Vihar Phase 3, New Delhi",3.9,183,4.1,545,https://www.zomato.com/ncr/bablu-chic-inn-asho...,"Shop 9, Pocket B, DDA Market, Ashok Vihar Phas...",9.2E+11,28.692473,77.178004,"Great Recommendations, Tamper Proof Packaging,...",NaN
1961,Kovilakam,South Indian,350,"Dilshad Garden, New Delhi",3.9,114,NaN,1,https://www.zomato.com/ncr/kovilakam-dilshad-g...,"F-1, Lower Ground 2, Mrignaini Chowk, Dilshad ...",9.19654E+11,28.684101,77.326725,"Kerala Parotta, Roasted Chicken, Chicken Curry...","Lip Smacking Food, Amazing Food, Owner, Good P..."
1962,Krips Restaurant,"North Indian, Chinese, Mughlai",900,"Basant Lok Market, Vasant Vihar, New Delhi",3.9,316,3.9,2977,https://www.zomato.com/ncr/krips-restaurant-va...,"59, Community Centre, Basant Lok Market, Vasan...",9.11141E+11,28.557310,77.163943,"Excellent Food, Good Ambience, Courteous Staff...",NaN
1963,Cafe 147,"Cafe, North Indian, Chinese, Kashmiri, Rolls, ...",800,"Sainik Farms, New Delhi",3.9,120,3.8,196,https://www.zomato.com/ncr/cafe-147-sainik-far...,"Shop A1/7, Ground Floor, Main IGNOU Road, Neb ...",9.19812E+11,28.510338,77.202056,"Pool, Live Music, Cafe, Environment, Good Plac...",NaN


In [24]:
print(data.isnull().sum())

Restaurant_Name            0
Category                   0
Pricing_for_2              0
Locality                   0
Dining_Rating              0
Dining_Review_Count        0
Delivery_Rating          402
Delivery_Rating_Count      0
Website                    0
Address                    0
Phone_No                   0
Latitude                   0
Longitude                  0
Known_For2               405
Known_For22              841
dtype: int64


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1965 entries, 0 to 1964
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Restaurant_Name        1965 non-null   object 
 1   Category               1965 non-null   object 
 2   Pricing_for_2          1965 non-null   int64  
 3   Locality               1965 non-null   object 
 4   Dining_Rating          1965 non-null   float64
 5   Dining_Review_Count    1965 non-null   int64  
 6   Delivery_Rating        1563 non-null   float64
 7   Delivery_Rating_Count  1965 non-null   int64  
 8   Website                1965 non-null   object 
 9   Address                1965 non-null   object 
 10  Phone_No               1965 non-null   object 
 11  Latitude               1965 non-null   float64
 12  Longitude              1965 non-null   float64
 13  Known_For2             1560 non-null   object 
 14  Known_For22            1124 non-null   object 
dtypes: f

In [26]:
data.describe()

,Pricing_for_2,Dining_Rating,Dining_Review_Count,Delivery_Rating,Delivery_Rating_Count,Latitude,Longitude
count,1965.000000,1965.000000,1965.000000,1563.000000,1965.000000,1965.000000,1965.000000
mean,1090.687023,4.140000,721.954198,3.964299,3029.550636,28.549542,77.161570
std,934.481617,0.202147,932.618839,0.242305,6529.839896,0.784241,1.685399
min,100.000000,3.900000,3.000000,2.800000,0.000000,-5.975200,2.650463
25%,500.000000,4.000000,244.000000,3.800000,34.000000,28.496564,77.100745
50%,700.000000,4.100000,433.000000,4.000000,604.000000,28.567882,77.201558
75%,1500.000000,4.300000,842.000000,4.100000,3316.000000,28.635845,77.258408
max,8000.000000,4.900000,10100.000000,4.500000,87700.000000,29.065189,77.536009


In [27]:
data=data.drop(['Dining_Review_Count','Delivery_Rating','Delivery_Rating_Count','Website','Address','Phone_No','Latitude','Longitude','Known_For2'],axis=1)

In [28]:
data

,Restaurant_Name,Category,Pricing_for_2,Locality,Dining_Rating,Known_For22
0,Rustom's,"Parsi, Street Food, North Indian, Desserts, Be...",2100,"ITO, New Delhi",4.9,NaN
1,Cafe Lota,"Cafe, South Indian, North Indian, Beverages",1200,"Pragati Maidan, New Delhi",4.9,"Artistic Decor, The Service, Natural Ambience,..."
2,Dum-Pukht - ITC Maurya,"Mughlai, North Indian, Desserts",5000,"ITC Maurya, Chanakyapuri, New Delhi",4.9,NaN
3,Burma Burma,"Asian, Burmese, Bubble Tea, Desserts, Salad",1600,"Cyber Hub, DLF Cyber City, Gurgaon",4.9,"Knowledgeable Staff, Authentic, Soothing Ambie..."
4,The Big Chill,"Continental, American, Italian",1500,"Khan Market, New Delhi",4.9,"Retro Ambience, Yummy Desserts, Big Portions, ..."
...,...,...,...,...,...,...
1960,Bablu Chic-Inn,"Mughlai, North Indian, Chinese, Fast Food",500,"Ashok Vihar Phase 3, New Delhi",3.9,NaN
1961,Kovilakam,South Indian,350,"Dilshad Garden, New Delhi",3.9,"Lip Smacking Food, Amazing Food, Owner, Good P..."
1962,Krips Restaurant,"North Indian, Chinese, Mughlai",900,"Basant Lok Market, Vasant Vihar, New Delhi",3.9,NaN
1963,Cafe 147,"Cafe, North Indian, Chinese, Kashmiri, Rolls, ...",800,"Sainik Farms, New Delhi",3.9,NaN


In [29]:
data.duplicated().sum()
data.drop_duplicates(inplace=True)
data.head()

,Restaurant_Name,Category,Pricing_for_2,Locality,Dining_Rating,Known_For22
0,Rustom's,"Parsi, Street Food, North Indian, Desserts, Be...",2100,"ITO, New Delhi",4.9,NaN
1,Cafe Lota,"Cafe, South Indian, North Indian, Beverages",1200,"Pragati Maidan, New Delhi",4.9,"Artistic Decor, The Service, Natural Ambience,..."
2,Dum-Pukht - ITC Maurya,"Mughlai, North Indian, Desserts",5000,"ITC Maurya, Chanakyapuri, New Delhi",4.9,NaN
3,Burma Burma,"Asian, Burmese, Bubble Tea, Desserts, Salad",1600,"Cyber Hub, DLF Cyber City, Gurgaon",4.9,"Knowledgeable Staff, Authentic, Soothing Ambie..."
4,The Big Chill,"Continental, American, Italian",1500,"Khan Market, New Delhi",4.9,"Retro Ambience, Yummy Desserts, Big Portions, ..."


In [30]:
data.isnull().sum()

Restaurant_Name      0
Category             0
Pricing_for_2        0
Locality             0
Dining_Rating        0
Known_For22        841
dtype: int64

In [31]:
data.dropna(how='any',inplace=True)
data.head()

,Restaurant_Name,Category,Pricing_for_2,Locality,Dining_Rating,Known_For22
1,Cafe Lota,"Cafe, South Indian, North Indian, Beverages",1200,"Pragati Maidan, New Delhi",4.9,"Artistic Decor, The Service, Natural Ambience,..."
3,Burma Burma,"Asian, Burmese, Bubble Tea, Desserts, Salad",1600,"Cyber Hub, DLF Cyber City, Gurgaon",4.9,"Knowledgeable Staff, Authentic, Soothing Ambie..."
4,The Big Chill,"Continental, American, Italian",1500,"Khan Market, New Delhi",4.9,"Retro Ambience, Yummy Desserts, Big Portions, ..."
5,Carnatic Cafe,South Indian,600,"Lodhi Colony, New Delhi",4.9,"Cosy Ambience, Live Kitchen, Quiet Place, Calm..."
6,"Cocktails & Dreams, Speakeasy","Nepalese, Tibetan, Beverages",2500,"Sector 15, Gurgaon",4.9,"Jazz Music, Band Playing, Bartender, Live Band..."


In [32]:
data = data.rename(columns={'Restaurant_Name':'name','Category':'cuisines','Pricing_for_2':'cost','Locality':'location', 'Dining_Rating':'rating', 'Known_For22':'reviews_list'})

In [33]:
data.head()

,name,cuisines,cost,location,rating,reviews_list
1,Cafe Lota,"Cafe, South Indian, North Indian, Beverages",1200,"Pragati Maidan, New Delhi",4.9,"Artistic Decor, The Service, Natural Ambience,..."
3,Burma Burma,"Asian, Burmese, Bubble Tea, Desserts, Salad",1600,"Cyber Hub, DLF Cyber City, Gurgaon",4.9,"Knowledgeable Staff, Authentic, Soothing Ambie..."
4,The Big Chill,"Continental, American, Italian",1500,"Khan Market, New Delhi",4.9,"Retro Ambience, Yummy Desserts, Big Portions, ..."
5,Carnatic Cafe,South Indian,600,"Lodhi Colony, New Delhi",4.9,"Cosy Ambience, Live Kitchen, Quiet Place, Calm..."
6,"Cocktails & Dreams, Speakeasy","Nepalese, Tibetan, Beverages",2500,"Sector 15, Gurgaon",4.9,"Jazz Music, Band Playing, Bartender, Live Band..."


In [34]:
data.name = data.name.apply(lambda x:x.title())

In [35]:
data['cost'] = data['cost'].astype(float)

In [36]:
restaurants = list(data['name'].unique())
data['Mean Rating'] = 0

for i in range(len(restaurants)):
    data['Mean Rating'][data['name'] == restaurants[i]] = data['rating'][data['name'] == restaurants[i]].mean()

In [37]:
data

,name,cuisines,cost,location,rating,reviews_list,Mean Rating
1,Cafe Lota,"Cafe, South Indian, North Indian, Beverages",1200.0,"Pragati Maidan, New Delhi",4.9,"Artistic Decor, The Service, Natural Ambience,...",4.9
3,Burma Burma,"Asian, Burmese, Bubble Tea, Desserts, Salad",1600.0,"Cyber Hub, DLF Cyber City, Gurgaon",4.9,"Knowledgeable Staff, Authentic, Soothing Ambie...",4.9
4,The Big Chill,"Continental, American, Italian",1500.0,"Khan Market, New Delhi",4.9,"Retro Ambience, Yummy Desserts, Big Portions, ...",4.9
5,Carnatic Cafe,South Indian,600.0,"Lodhi Colony, New Delhi",4.9,"Cosy Ambience, Live Kitchen, Quiet Place, Calm...",4.9
6,"Cocktails & Dreams, Speakeasy","Nepalese, Tibetan, Beverages",2500.0,"Sector 15, Gurgaon",4.9,"Jazz Music, Band Playing, Bartender, Live Band...",4.9
...,...,...,...,...,...,...,...
1952,Aman-Deep Pure Veg,"North Indian, Chinese",400.0,"Shalimar Bagh, New Delhi",3.9,"Seating, Worth the Money, Prices, Timely Servi...",3.9
1953,All Biryani King,"Mughlai, North Indian",400.0,"Uttam Nagar, New Delhi",3.9,"Variety of Pizza, Customizable Dishes, Weekend...",3.9
1955,Chilli Dragon,"Chinese, Tibetan, Thai, Seafood",700.0,"Malviya Nagar, New Delhi",3.9,"Casual Seating Area, Chinese Restaurant, Nice ...",3.9
1956,Playboy Tap,"European, Continental",1600.0,"Global Foyer Mall, Golf Course Road, Gurgaon",3.9,"Live Entertainment, Good Bartenders, Beautiful...",3.9


In [38]:
data.loc[data.name =='Burma Burma']

,name,cuisines,cost,location,rating,reviews_list,Mean Rating
3,Burma Burma,"Asian, Burmese, Bubble Tea, Desserts, Salad",1600.0,"Cyber Hub, DLF Cyber City, Gurgaon",4.9,"Knowledgeable Staff, Authentic, Soothing Ambie...",4.9


In [39]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (1,5))
data[['Mean Rating']] = scaler.fit_transform(data[['Mean Rating']]).round(2)

In [40]:
data.head()

,name,cuisines,cost,location,rating,reviews_list,Mean Rating
1,Cafe Lota,"Cafe, South Indian, North Indian, Beverages",1200.0,"Pragati Maidan, New Delhi",4.9,"Artistic Decor, The Service, Natural Ambience,...",5.0
3,Burma Burma,"Asian, Burmese, Bubble Tea, Desserts, Salad",1600.0,"Cyber Hub, DLF Cyber City, Gurgaon",4.9,"Knowledgeable Staff, Authentic, Soothing Ambie...",5.0
4,The Big Chill,"Continental, American, Italian",1500.0,"Khan Market, New Delhi",4.9,"Retro Ambience, Yummy Desserts, Big Portions, ...",5.0
5,Carnatic Cafe,South Indian,600.0,"Lodhi Colony, New Delhi",4.9,"Cosy Ambience, Live Kitchen, Quiet Place, Calm...",5.0
6,"Cocktails & Dreams, Speakeasy","Nepalese, Tibetan, Beverages",2500.0,"Sector 15, Gurgaon",4.9,"Jazz Music, Band Playing, Bartender, Live Band...",5.0


In [41]:
#Before Text Processing
data[['reviews_list', 'cuisines']].head()

,reviews_list,cuisines
1,"Artistic Decor, The Service, Natural Ambience,...","Cafe, South Indian, North Indian, Beverages"
3,"Knowledgeable Staff, Authentic, Soothing Ambie...","Asian, Burmese, Bubble Tea, Desserts, Salad"
4,"Retro Ambience, Yummy Desserts, Big Portions, ...","Continental, American, Italian"
5,"Cosy Ambience, Live Kitchen, Quiet Place, Calm...",South Indian
6,"Jazz Music, Band Playing, Bartender, Live Band...","Nepalese, Tibetan, Beverages"


In [ ]:
#Lower Casing
data["reviews_list"] = data["reviews_list"].str.lower()

#Removal of Punctuations 
import string
punc = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punc))

data["reviews_list"] = data["reviews_list"].apply(lambda text: remove_punctuation(text))

#Removal of Stopwords
Stopwords = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in Stopwords])

data["reviews_list"] = data["reviews_list"].apply(lambda text: remove_stopwords(text))

#Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

data["reviews_list"] = data["reviews_list"].apply(lambda text: remove_urls(text))
data[['reviews_list', 'cuisines']].head()

# Recommendation by Reviews

In [42]:
data

,name,cuisines,cost,location,rating,reviews_list,Mean Rating
1,Cafe Lota,"Cafe, South Indian, North Indian, Beverages",1200.0,"Pragati Maidan, New Delhi",4.9,"Artistic Decor, The Service, Natural Ambience,...",5.0
3,Burma Burma,"Asian, Burmese, Bubble Tea, Desserts, Salad",1600.0,"Cyber Hub, DLF Cyber City, Gurgaon",4.9,"Knowledgeable Staff, Authentic, Soothing Ambie...",5.0
4,The Big Chill,"Continental, American, Italian",1500.0,"Khan Market, New Delhi",4.9,"Retro Ambience, Yummy Desserts, Big Portions, ...",5.0
5,Carnatic Cafe,South Indian,600.0,"Lodhi Colony, New Delhi",4.9,"Cosy Ambience, Live Kitchen, Quiet Place, Calm...",5.0
6,"Cocktails & Dreams, Speakeasy","Nepalese, Tibetan, Beverages",2500.0,"Sector 15, Gurgaon",4.9,"Jazz Music, Band Playing, Bartender, Live Band...",5.0
...,...,...,...,...,...,...,...
1952,Aman-Deep Pure Veg,"North Indian, Chinese",400.0,"Shalimar Bagh, New Delhi",3.9,"Seating, Worth the Money, Prices, Timely Servi...",1.0
1953,All Biryani King,"Mughlai, North Indian",400.0,"Uttam Nagar, New Delhi",3.9,"Variety of Pizza, Customizable Dishes, Weekend...",1.0
1955,Chilli Dragon,"Chinese, Tibetan, Thai, Seafood",700.0,"Malviya Nagar, New Delhi",3.9,"Casual Seating Area, Chinese Restaurant, Nice ...",1.0
1956,Playboy Tap,"European, Continental",1600.0,"Global Foyer Mall, Golf Course Road, Gurgaon",3.9,"Live Entertainment, Good Bartenders, Beautiful...",1.0


In [43]:
df_percent = data.sample(frac=1)
df_percent.set_index('name', inplace=True)
indices = pd.Series(df_percent.index)

In [44]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [45]:
# def recommend_like(name, top_n=10, cosine_similarities=cosine_similarities):
#     recommend_restaurant = []
#     try:
#         idx = indices[indices == name].index[0]
#         score_series = pd.Series(cosine_similarities[idx])
#         top_indexes = list(score_series.iloc[1:top_n+1].index)

#         for each in top_indexes:
#             recommend_restaurant.append(list(df_percent.index)[each])

#         df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])

#         for each in recommend_restaurant:
#             selected_data = df_percent[['cuisines', 'Mean Rating', 'cost']][df_percent.index == each].sample()
#             df_new = pd.concat([df_new, selected_data])

#         df_new = df_new.drop_duplicates(subset=['cuisines', 'Mean Rating', 'cost'], keep=False)
#         df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(top_n)

#         print(f'TOP {top_n} RESTAURANTS LIKE {name.upper()}: ')
#         return df_new
#     except:
#         print('No restaurants with similar reviews.')

def recommend_like(name, top_n=10, cosine_similarities=cosine_similarities):
    recommend_restaurant = []
    try:
        idx = indices[indices == name].index[0]
        score_series = pd.Series(cosine_similarities[idx])
        top_indexes = list(score_series.iloc[1:top_n+1].index)

        for each in top_indexes:
            recommend_restaurant.append(list(df_percent.index)[each])

        df_new = pd.DataFrame(columns=['name', 'cuisines', 'Mean Rating', 'cost'])
        for each in recommend_restaurant:
            # selected_data = df_percent.loc[df_percent.index == each, ['cuisines', 'Mean Rating', 'cost']].sample()
            selected_data = df_percent.loc[df_percent.index == each, ['cuisines', 'Mean Rating', 'cost']].sample()
            selected_data['name'] = each 
            df_new = pd.concat([df_new, selected_data])
            # df_new = pd.concat([df_new, selected_data.reset_index(drop=True)])
            # df_new = pd.concat([df_new, selected_data], ignore_index=True)
        df_new = df_new.drop_duplicates(subset=[ 'name','cuisines', 'Mean Rating', 'cost'], keep=False)
        df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(top_n)
        # df_new = df_new.loc[:, ~df_new.columns.str.contains('^Unnamed')]
        print(f'TOP RESTAURANTS LIKE {name.upper()}: ')
        # df_new.style.hide_index()
        # return df_new
        df_new.reset_index(drop=True, inplace=True)
        return df_new
    except:
        print('No restaurants with similar reviews.')


In [46]:
# df_percent

In [47]:
recommend_like('The Big Chill')

TOP RESTAURANTS LIKE THE BIG CHILL: 


,name,cuisines,Mean Rating,cost
0,Concept Shawarma,"North Indian, Chinese",2.6,650.0
1,Bamboo Shoots,Asian,2.2,1600.0
2,The Royal Kitchen,"North Indian, Chinese, Mughlai, Biryani",2.2,700.0
3,Subway,"Healthy Food, Fast Food, Salad, Wraps",1.8,500.0
4,La Vie Pizzeria,"Pizza, Fast Food, Beverages",1.8,1000.0
5,Mocha Arthouse,"Cafe, North Indian, Continental, Beverages, De...",1.8,1400.0
6,Rajdhani,"Gujarati, Rajasthani, North Indian",1.8,1100.0
7,Ocean Blue,"North Indian, Chinese, Street Food, Italian, F...",1.4,1100.0
8,Desi We Desi,"North Indian, Chinese",1.4,500.0
9,Pizza And More,"Pizza, Fast Food",1.0,400.0


# Recommendation by Cuisine

In [57]:
 #df_percent = data.sample(frac=1)
 #df_percent.set_index('cuisines'.lower(), inplace=True) 
 #indices = pd.Series(df_percent.index)

 #from fuzzywuzzy import fuzz

 Create a mapping of normalized cuisine values to restaurant names
 # cuisine_mapping = {}
 #for index, row in data.iterrows():
 #    for cuisine in row['cuisines'].split(', '):
#         normalized_cuisine = cuisine.lower()
#         if normalized_cuisine not in cuisine_mapping:
#             cuisine_mapping[normalized_cuisine] = []
#         cuisine_mapping[normalized_cuisine].append(index)

# def recommend_by_cuisine(criteria_value, top_n=10, cosine_similarities=cosine_similarities):
#     recommend_restaurant = []
#     normalized_criteria = criteria_value.lower()
#     matching_restaurants = cuisine_mapping.get(normalized_criteria, [])

#     for restaurant_index in matching_restaurants:
#         recommend_restaurant.append(restaurant_index)

#     df_new = pd.DataFrame(columns=['name', 'cuisines', 'Mean Rating', 'cost'])
#     for each in recommend_restaurant:
#         selected_data = data[['name', 'cuisines', 'Mean Rating', 'cost']][data.index == each].sample()
#         df_new = pd.concat([df_new, selected_data])
#     df_new = df_new.drop_duplicates(subset=['name', 'cuisines', 'Mean Rating', 'cost'], keep=False)
#     df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(top_n)

    # print(f'TOP {top_n} RESTAURANTS WITH SIMILAR CUISINE: ')
   #  df_new.set_index('name', inplace=True)
  #  return df_new


def recommend_by_cuisine(criteria_value, top_n=10, cosine_similarities=cosine_similarities):
    recommend_restaurant = []
    normalized_criteria = criteria_value.lower()
    matching_restaurants = cuisine_mapping.get(normalized_criteria, [])

    for restaurant_index in matching_restaurants:
        recommend_restaurant.append(restaurant_index)

    df_new = pd.DataFrame(columns=['name', 'cuisines', 'Mean Rating', 'cost'])
    for each in recommend_restaurant:
        selected_data = data[['name', 'cuisines', 'Mean Rating', 'cost']][data.index == each].sample()
        df_new = pd.concat([df_new, selected_data])
    df_new = df_new.drop_duplicates(subset=['name', 'cuisines', 'Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(top_n)

    print(f'TOP {top_n} RESTAURANTS WITH SIMILAR CUISINE: ')
    # df_new.set_index('name', inplace=True)
    df_new.reset_index(drop=True, inplace=True)

In [ ]:
recommend_by_cuisine('Chinese')

# Recommendation by Rating

In [53]:
df_percent = data.sample(frac=1)
df_percent.set_index('Mean Rating', inplace=True) 
indices = pd.Series(df_percent.index)

def recommend_by_mean_rating(mean_rating_value, top_n=10):
    recommend_restaurant = data[data['Mean Rating'] >= mean_rating_value].index
    df_new = data.loc[recommend_restaurant, ['name', 'cuisines', 'Mean Rating', 'cost']]
    df_new = df_new.sample(frac=1) 
    df_new = df_new.head(top_n)
    print(f'TOP {top_n} RESTAURANTS WITH A MINIMUM RATING OF {mean_rating_value}: ')
    df_new.set_index('name', inplace=True)
    return df_new

In [54]:
recommend_by_mean_rating(4)

TOP 10 RESTAURANTS WITH A MINIMUM RATING OF 4: 


,cuisines,Mean Rating,cost
name,,,
Cafe Lota,"Cafe, South Indian, North Indian, Beverages",5.0,1200.0
Andrea'S Eatery,"Asian, Italian, Beverages, Continental, Fast F...",4.2,2000.0
Chili'S Grill & Bar,"Mexican, Italian, American, Tex-Mex, Beverages...",4.2,1200.0
Threesixtyone Degrees - The Oberoi,"Asian, Continental, North Indian, Mughlai, Des...",4.2,5500.0
Olive Bar & Kitchen,"Italian, Pizza, Mediterranean, Continental, Eu...",4.2,4000.0
Cafe Wink,"Cafe, Continental, Italian",4.2,800.0
The Big Chill Cakery,"Bakery, Desserts",4.6,400.0
"Cocktails & Dreams, Speakeasy","Nepalese, Tibetan, Beverages",5.0,2500.0
Downtown - Diners & Living Beer Cafe,"Bar Food, North Indian, Fast Food, Italian, Ch...",4.2,2200.0


# Recommendation by Cost

In [55]:
df_percent = data.sample(frac=1)
df_percent.set_index('cost', inplace=True)  
indices = pd.Series(df_percent.index)

def recommend_by_cost(min_value,max_value, top_n=10):
    recommend_restaurant = data[(data['cost'] > min_value) & (data['cost'] <= max_value)].index
    df_new = data.loc[recommend_restaurant, ['name', 'cuisines', 'Mean Rating', 'cost']]
    df_new = df_new.sample(frac=1) 
    df_new = df_new.head(top_n)
    print(f'TOP {top_n} RESTAURANTS IN THE PROVIDED RANGE: ')
    df_new.set_index('name', inplace=True)
    return df_new

In [56]:
recommend_by_cost(3000,4000)

TOP 10 RESTAURANTS IN THE PROVIDED RANGE: 


,cuisines,Mean Rating,cost
name,,,
Larry'S China - Ambassador,"Chinese, Seafood, Desserts",1.4,4000.0
Tonino,"Italian, Pizza, Pasta, Salad, Beverages",3.4,3500.0
Olive Bar & Kitchen,"Italian, Pizza, Mediterranean, Continental, Eu...",4.2,4000.0
Cascades - The Grand New Delhi,"North Indian, Continental, Asian",1.0,3200.0
Elan - The Lodhi,"North Indian, Continental, Italian, Salad",1.8,3500.0
Spice Art - Crowne Plaza,"North Indian, Awadhi, Kebab, Mughlai, Asian, B...",3.0,3500.0
Rice - Radisson Blu Dwarka,"Chinese, Thai, Vietnamese, Japanese",2.2,4000.0
Frontier - The Ashok,"North Indian, Mughlai",1.4,3600.0
Paatra - Jaypee Greens Golf & Spa Resort,North Indian,1.4,3500.0
